<a href="https://colab.research.google.com/github/Sompote/COV_pile/blob/main/GBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#bearing capacity by Dr. Sompote 2021
from math import *
import numpy as np
import pandas as pd

In [ ]:
def bearing(phi):
  if phi==0:
    Nc=5.14
    Nq=1
    Ny=0
  else: 
    ϕ=phi*(np.pi/180)
    ϕ1=(45+(phi/2))*(pi/180)
    Nq=((np.tan(ϕ1))**2)*(np.e**(pi*tan(ϕ)))
    Ny=2*(Nq+1)*tan(ϕ)
    Nc = (Nq-1)/tan(ϕ)
  return Nc, Nq, Ny

def factor(phi,Nc, Nq,B,L,D,beta):
  #Shape factor (Debeer,1970)
  
  ϕ =(phi/180*pi)
  Fcs =1+ (B/L)*(Nq/Nc) 
  Fqs =1+(B/L)*tan(ϕ)
  Fγs = 1-0.4*(B/L) 

  #Depth factor
  if (D/B) > 1:
    if (phi ==0):
      Fcd = 1+0.4*atan(D/B)
      Fqd=1
      Fγd = 1
    else:
      Fγd = 1
      Fqd =1+(2*tan(ϕ)*(1-(sin(ϕ))**2)*atan(D/B))
      Fcd = Fqd - ((1-Fqd)/(Nc*tan(ϕ)))
  else:#(D/B)<1:
      if (phi ==0) :
        Fcd = 1+0.4*(D/B)
        Fqd=1
        Fγd = 1
      else: 
        Fqd= 1+2*tan(ϕ)*(1-sin(ϕ))**2*(D/B)
        Fcd=Fqd-(1-Fqd)/(Nc*tan(ϕ))
        Fγd = 1
  #Inclination
  Fci = (1-(beta/90))**2                       # Meyerhof (1963) ; Hanna and
  Fqi =(1-(beta/90))**2                        # Meyerhof (1981)
  if phi>0:
    Fyi = (1-(beta/phi))**2
  else:
    Fyi= 1
  return Fcs, Fqs, Fγs, Fcd, Fqd, Fγd, Fci, Fqi, Fyi
#Paramters
fee =25
gam=16.5
c=20
B=2
L=2
D=1.5
beta=0 #inclinaiton factor
q=D*gam
#calculaiton
Nc, Nq,Ng=bearing(fee)
fcs,fqs,fgs,fcd,fqd,fgd,fci,fqi,fgi=factor(fee,bearing(fee)[0],
                                           bearing(fee)[1], B, L, D,beta)
qult=c*Nc*(fcs*fcd*fci)+q*Nq*(fqs*fqd*fqi)+0.5*gam*B*Ng*(fgs*fgd*fgi)
print('qult=',qult, 'kPa')

#show value add factor in dataframe
fac=np.array(bearing(fee))
fac=fac.reshape(1,3)
df1=pd.DataFrame(fac,columns=['Nc', 'Nq', 'Ng'])
print(df1)
fac_adjust=np.array(factor(fee,bearing(fee)[0],bearing(fee)[1], B, L, D,beta))
fac_adjust=fac_adjust.reshape(1,9)
df=pd.DataFrame(fac_adjust,columns=(['Fcs', 'Fqs', 'Fgs',
                               'Fcd','Fqd','Fgd','Fci','Fqi','Fgi']))
df

qult= 1373.9995219835032 kPa
          Nc         Nq         Ng
0  20.720531  10.662142  10.876293


,Fcs,Fqs,Fgs,Fcd,Fqd,Fgd,Fci,Fqi,Fgi
0,1.514569,1.466308,0.6,1.257313,1.233179,1.0,1.0,1.0,1.0
